In [2]:
import csv
import pandas as pd
import numpy as np


# Generate Jira Dataset

In [5]:
def emotionToSentiment():    
    group1 = ['Document1', 'Document2', 'Document3', 'Document4', 'Document5', 'Document6', 'Document7', 'Document8', 
              'Document9', 'Document10', 'Document11', 'Document12', 'Document13', 'Document14', 'Document15', 'Document16']

    group2 = ['emoiton_1', 'emoiton_2', 'emoiton_3']

    group3 = ['SentenceWithEmotion_TRAINING_ANGER', 
              'SentenceWithEmotion_TRAINING_JOY', 
              'SentenceWithEmotion_TRAINING_LOVE',
              'SentenceWithEmotion_TRAINING_SADNESS']

    groups = {
                '1': group1, 
                '2': group2, 
                '3': group3
            }

    ids = [] 
    d = {}

    for key, gr in groups.items():
        for doc in gr:
            row = pd.read_csv('Datasets/GroupsJiraDataset/group' + key + '/' + doc + '.csv', encoding = "ISO-8859-1", engine='python')
            #print(row.shape)
            for k, r in row.iterrows():
                comKey = 'comment N'
                if 'comment' in row.columns.values:
                    comKey = 'comment' 

                s = {'love': str(r['love']), 'joy': str(r['joy']), 'surprise': str(r['surprise']), 'anger': str(r['anger']), 'sadness': str(r['sadness']), 'fear': str(r['fear'])}

                if str(r[comKey]) == 'nan':
                    continue

                if str(r['id']) in ids:
                    d[str(r['id'])]['Label'].append(s)
                    continue
                else:
                    d[str(r['id'])] = {'Sentence': str(r[comKey]), 'Label': [s], 'group': key}

                ids.append(str(r['id']))

    emoSurprise = []
    for key, row in d.items():
        emo = {'love': 0, 'joy': 0, 'surprise': 0, 'anger': 0, 'sadness': 0, 'fear': 0}
        for value in row['Label']:
            if value['love'] == 'x':
                emo['love'] += 1
            if value['joy'] == 'x':
                emo['joy'] += 1
            if value['surprise'] == 'x':
                emo['surprise'] += 1
            if value['anger'] == 'x':
                emo['anger'] += 1
            if value['sadness'] == 'x':
                emo['sadness'] += 1
            if value['fear'] == 'x':
                emo['fear'] += 1

        pos = emo['love']  + emo['joy']
        neg = emo['anger'] + emo['sadness'] + emo['fear']
        emot = max(emo, key=emo.get)
        sent = '0'

        if pos > neg and ((pos != 1) or row['group'] == '3'):
            sent = '1'
        if neg > pos and ((neg != 1) or row['group'] == '3'):
            sent = '-1'
        if emo['surprise'] > 0 and sent == '0':
            emoSurprise.append(key)

        d[key]['Label'] = sent    
        #print(key, 'g'+row['group'], emo['love'], emo['joy'], emo['surprise'], emo['anger'], emo['sadness'], emo['fear'], '-->', sent)

    for i in emoSurprise:
        del d[i]

    #print(len(emoSurprise))
    datasetJira = pd.DataFrame(d).transpose() 
    print(datasetJira.shape)
    print(datasetJira['Label'].value_counts('1')*100)
    datasetJira.head()           

    
emotionToSentiment()  

(5888, 3)
0     66.083560
1     19.921875
-1    13.994565
Name: Label, dtype: float64


In [193]:
# Save dataset
datasetJira.to_csv('Datasets/datasetJIRA.csv', encoding='utf-8', header=True, index=False)
